# Prepare the Environment


In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
from collections import Counter, OrderedDict
import statistics
import operator

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import scipy.stats as ss

import xgboost
import pickle

from scipy.stats import norm
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from pandas import read_csv

from sklearn.ensemble import IsolationForest
import sklearn.metrics

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso

# flag for verbose printout of lengthy functions
verbose = False

# flag for submission file creation
submission = False

In [ ]:
pd.set_option('max_columns', None)

## Function to create submission file

In [ ]:
def submission_file(y_pred, filename):

  pred=pd.DataFrame(y_pred)
  sub_df=pd.read_csv('https://raw.githubusercontent.com/rambles-tech/w207_final_project_kaggle/main/data/sample_submission.csv')

  # grab just the ID from sample_submission.csv
  datasets=pd.concat([sub_df['Id'],pred],axis=1)
  
  # append y_pred to newly-created DF
  datasets.columns=['Id','SalePrice']
  
  # write to file
  datasets.to_csv(filename,index=False)

# Bring in the Data

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/rambles-tech/w207_final_project_kaggle/main/data/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/rambles-tech/w207_final_project_kaggle/main/data/test.csv")

# combine train and test data for ease of feature engineering
data = pd.concat([train_data, test_data]) #1459 is last training_data entry


# EDA

Confirm the shape of the data  
*Note: test_data should have one less column (sale_price)*

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(data.shape)

## Validate Data

The following dictionary is created from the original data description and inference following domain-specific research.  This will be used to check for inconsistent or incorrect values in the dataset.

In [ ]:
# feature values
feature_values_dict = {
    ### NUMERIC FEATURES ###
    'LotFrontage': [0,1000000], 
    'LotArea' : [0,1000000], 
    'MasVnrArea' : [0, 10000], 
    'BsmtFinSF1' : [0, 10000], 
    'BsmtFinSF2' :[0, 10000], 
    'BsmtUnfSF' : [0, 10000], 
    'TotalBsmtSF' : [0, 10000], 
    '1stFlrSF' : [0, 10000], 
    '2ndFlrSF' : [0, 10000], 
    'LowQualFinSF' : [0, 10000], 
    'GrLivArea': [0, 10000], 
    'BsmtFullBath' : [0, 10],
    'BsmtHalfBath' : [0, 10],
    'FullBath' : [0, 10], 
    'HalfBath' : [0, 10], 
    'BedroomAbvGr' : [0, 10], 
    'KitchenAbvGr' : [0, 10], 
    'TotRmsAbvGrd' : [0, 20], 
    'Fireplaces' : [0, 5], 
    'GarageCars' : [0,10000], 
    'GarageArea': [0, 10000], 
    'WoodDeckSF' : [0, 2000], 
    'OpenPorchSF' : [0, 2000], 
    'EnclosedPorch' : [0, 2000], 
    '3SsnPorch' : [0, 2000], 
    'ScreenPorch' : [0, 2000], 
    'PoolArea' : [0, 2000], 
    'MiscVal' : [0, 50000], 
    'SalePrice' : [0,1000000],

    ### CATEGORICAL and ORDINAL FEATURES ###
    'Id' : [1,2919],
    'MSSubClass': [20,30,40,45,50,60,70,75,80,85,90,120,150,160,180,190],
    'MSZoning': ['A','C','FV', 'I', 'RH','RL', 'RP', 'RM'], 
    'Street' : ['Grvl', 'Pave'], 
    'Alley' : ['Grvl', 'Pave', 'NA'], 
    'LotShape' : ['Reg', 'IR1', 'IR2','IR3'], 
    'LandContour' : ['Lvl', 'Bnk', 'HLS', 'Low'], 
    'Utilities' : ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], 
    'LotConfig' : ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'], 
    'LandSlope': ['Gtl', 'Mod', 'Sev'], 
    'Neighborhood' : ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 
                      'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR',
                      'MeadowV', 'Mitchel', 'Names', 'NoRidge', 'NPkVill', 
                      'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 
                      'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'], 
    'Condition1' : ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], 
    'Condition2' : ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], 
    'BldgType' : ['1Fam', '2FmCon', 'Duplx', 'TwnhsE', 'TwnhsI'],
    'HouseStyle' : ['1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl'], 
    'OverallQual' : [1,2,3,4,5,6,7,8,9,10], 
    'OverallCond' : [1,2,3,4,5,6,7,8,9,10], 
    'YearBuilt' : [1850,2010], 
    'YearRemodAdd' : [1850,2010], 
    'RoofStyle' : ['Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'], 
    'RoofMatl' : ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'], 
    'Exterior1st' : ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock' , 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'], 
    'Exterior2nd' : ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock' , 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'],
    'MasVnrType' : ['BrkCmn','BrkFace','CBlock','None','Stone'], 
    'ExterQual' : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], 
    'ExterCond' : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], 
    'Foundation' : ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], 
    'BsmtQual' : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 
    'BsmtCond' : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 
    'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'NA'], 
    'BsmtFinType1' : ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
    'BsmtFinType2' : ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'], 
    'Heating' : ['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'], 
    'HeatingQC' : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], 
    'CentralAir' : ['N', 'Y'], 
    'Electrical' : ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'], 
    'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'], 
    'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'], 
    'FireplaceQu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 
    'GarageType' : ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd', 'NA'], 
    'GarageYrBlt':[1850, 2010], 
    'GarageFinish' : ['Fin', 'RFn', 'Unf', 'NA'], 
    'GarageQual' : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 
    'GarageCond' : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 
    'PavedDrive' : ['Y', 'P', 'N'], 
    'PoolQC' : ['Ex', 'Gd', 'TA', 'Fa', 'NA'], 
    'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'], 
    'MiscFeature': ['Elev', 'Gar2', 'Othr', 'Shed', 'TenC', 'NA'], 
    'MoSold' : [1,2,3,4,5,6,7,8,9,10,11,12], 
    'YrSold' : [2006, 2007, 2008, 2009, 2010], 
    'SaleType' : ['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth'],
    'SaleCondition' : ['Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial']
}

This function will compare vaues of each feature in the dataset against the above-created dictionary of valid ranges/values.

In [ ]:
# examine the feature stats
def examine_feature(df,list_features,dict_values):
  for feature in list_features:
    print(df[feature].head())

    # create a dictionary of keys and values
    class_keys = Counter(df[feature]).keys()
    class_values = Counter(df[feature]).values()
    
    class_dict = dict(zip(class_keys,class_values))

    # sort dict by values
    od_values = dict(sorted(class_dict.items(), key=lambda item: item[1]))

    # variable to hold the count
    cnt = 0
    
    # list to hold visited values
    visited = []

    # loop for counting the unique values in feature
    for i in range(0, len(df[feature])):
      if df[feature].iloc[i] not in visited: 
        visited.append(df[feature].iloc[i])
        cnt += 1

    # sort the list of unique visited feature values
    visited.sort(key=repr)
    if isinstance(data[feature].iloc[0],np.int64) or isinstance(data[feature].iloc[0],np.float64):
      print('min =',df[feature].min())
      print('max =',df[feature].max())
      visited.sort()
      
    # print('mode =',df[feature].mode())
    print('value sort     :',od_values)
    print('# of samples   :',len(df))
    print('nans =',len(df[df[feature].isna()]))
    print('# of uniques   :',cnt)
    print('unique values  :',visited)

    print('possible values:',sorted(dict_values[feature]))
    print('cat not in list:',(list(set(visited)-set(dict_values[feature]))))
    print('cat to add as col:',(list(set(dict_values[feature])-set(visited))))
    print('*************')

In [ ]:
# examine feature values 
if verbose: examine_feature(data,data.columns,feature_values_dict)
else: print("Output suppressed; set `verbose` to True if output is desired")


## Identify Duplicate Entries

In [ ]:
# no duplicate id's
data.Id.unique().size

No duplicate data found

## Identify Missing Data  

In [ ]:
plt.figure(figsize=(24,8))
sns.heatmap(data.isnull(),yticklabels=False,cbar=False)
plt.show()

In [ ]:
# find NA values, sorted by columns with most NA entries
data.isnull().sum(0).sort_values(ascending=False).head(36)

## Identify Incorrect Data

### The Neighborhood Level

Lets compare values at the data-set level compared with the neighborhood level

In [ ]:
examples = ['LotFrontage', 'LotArea', 'YearBuilt']

fig, axes = plt.subplots(1, len(examples))

#plt.figure(figsize=(20,10)) 
fig.suptitle('Comparison between overall data mean (in red) and neighborhood means (in blue)')
fig.tight_layout()

for i, feature in enumerate(data[examples].columns):
  
  if data[feature].dtypes == 'int64' or data[feature].dtypes == 'float64' :
    axes[i].set_title(feature)
    # plot the mean by neighborhood
    axes[i].plot( data.groupby('Neighborhood')[feature].mean()  )

    # plot the overall mean
    axes[i].axhline(y = data[feature].value_counts().idxmax(), color = 'r', linestyle = '-') 

    axes[i].get_xaxis().set_visible(False)
  else:
    
    print(feature)
    print("Overall Mode")
    print(data[feature].value_counts().idxmax())

    print("Neighborhood Modes")
    print(data.groupby('Neighborhood')[feature].value_counts().idxmax())

plt.subplots_adjust(left=None, bottom=None, right=None, top=.8, wspace=.8, hspace=None)
plt.gcf().set_size_inches(10, 4)
plt.show()

**Recommendation: Considering the extreme changes by neighborhood, we recommend using means and modes at the neighborhood level when considering imputing missing values.**

In [ ]:
data[(data.Neighborhood == 'Blmngtn') & (data.LotArea == 3182)]

In [ ]:
# Check for standardized values at the neighborhood level
data[['Id','Neighborhood','MSSubClass','MSZoning']][data['Neighborhood'] == 'BrkSide']

There could potentially be some misclassified `MSSubClass` & `MSZoning` values.  Is it possible the same Neighborhood be both Residencial Medium (RM) and Low (RL) Density?  
**Recommendation: Conduct research to determine if houses within the same neighborhood could potentially have differing values.**



### The House Level

In [ ]:
data[(data.YearBuilt > 2010) | (data.YearRemodAdd > 2010)]

There are no houses built or remodeled after 2010.

In [ ]:
# Check for illogical sequences
data[['Id','YearBuilt','YearRemodAdd','Neighborhood']][data['YearBuilt'] > data['YearRemodAdd']]

Here, we found one case where `YearRemodAdd` was likely incorrectly entered prior to the `YearBuilt`.

**Recommendation: Drop or change this value for YearRemodAdd to 2002
or add a flat indictor variable for Remodeled**

In [ ]:
# no errors: houses with more than 400 square feet on 2nd floor than the 1st
temp_list = data['1stFlrSF'] - data['2ndFlrSF']
plt.hist(temp_list,bins=10)
plt.show()

df_temp = data[['Id','1stFlrSF','2ndFlrSF']]
df_temp[((df_temp['1stFlrSF'] - df_temp['2ndFlrSF']) < -400)].head(20)


It is unclear if this is an error or a factor of architectural design.    
**Recommend not changing anything, but certainly be aware of this issue if predictions are not in compliance.**

In [ ]:
# Check for potential misclassifications
df_temp = (data[(data.HouseStyle == '1Story') & 
                        (data['2ndFlrSF'] > 0)])
df_temp[['Id','MSSubClass','HouseStyle','1stFlrSF','2ndFlrSF']]


There are three houses which are misclassified as 1 story houses yet they have a 2nd floor.  
**Recommendation: Keep.  This could potentially cover a finished attic.  Since they are so few entries with this error, it would be unneccessary to drop these rows.**

### The Basement

In [ ]:
# counts match
print(data[(data.BsmtFinSF1 > 0)].shape[0])
print(data[(data.BsmtFinSF2 > 0)].shape[0])
#print(data[(data.BsmtFinSF1.isnull())])
data[['Id','BsmtFinSF1']].iloc[2120]

There does not seeom to be any discrepancies between the counts of BsmtFinSF1 and BsmtFinSF2.

There is at least one NaN value in this feature that must be addressed.

**Recommendation: Replace BsmtFinSF1 NaN with neighborhood mean.**



### The Garage

In [ ]:
# check for valid garage entries
data[['Id','YearBuilt','YearRemodAdd','GarageYrBlt']][(data.GarageYrBlt > 2010)]

There is one value that is clearly an entry error.  
**Recommendation: Either drop this entry, change to YearBuilt to match the house, change to 2007, or replace with neighborhood mean.**


In [ ]:
# find garages built before the houses
data[['Id','YearBuilt','GarageYrBlt']][(data.GarageYrBlt < data.YearBuilt)]

**Recommendation: Keep these values.  Without additional research and more data, it is likely not possible to rule these as errors and not a result of a house being demolished while the garage remains.**

### The Lot

In [ ]:
# Check various features of the lots
depth = (data.LotArea / data.LotFrontage)
df_temp = pd.DataFrame(data.Id)
df_temp['LotFrontage'] = data.LotFrontage
df_temp['depth'] = depth
df_temp['LotArea'] = data.LotArea
print(df_temp.max())
print(df_temp.min())
print(df_temp[(df_temp.depth < 75)].head())

Nothing significant to report.

In [ ]:
# check for lot plus building total
data[['Id','LotArea',
         '1stFlrSF','GarageArea','WoodDeckSF',
         'OpenPorchSF','EnclosedPorch','ScreenPorch',
         'PoolArea']][(data['1stFlrSF'] + data.GarageArea + data.WoodDeckSF + data.OpenPorchSF + data.EnclosedPorch + data.ScreenPorch + data.PoolArea)
          > data.LotArea]

There is one entry where the total areas of each component sum greater than the total lot - more square footage of buildings(plus) than the lot.  
**Recommendation: Although this entry should be dropped, since it a test subject, the value should be adjusted to equal the sum of components.**

In [ ]:
# Check for LotFrontAge Issues
data[['Id','LotFrontage','LotArea']][(data.LotFrontage < 24)].sort_values(by=['LotArea'], ascending=False).head()

Id 2039 LotFrontage and LotArea seem complex and likely is the result of an error.  
**Recommendation: This appears to be an extreme outlier, but it is in the test data.  Potentially, this value could be weighted to be more inline with the rest of the samples, but that would not achieve much of an effect.**


## Sales Price

In [ ]:
# examine the sales price data
round(train_data.SalePrice.describe(), 2)

In [ ]:
(mu, sigma) = norm.fit(train_data['SalePrice'])
plt.figure(figsize = (12,6))
sns.distplot(train_data['SalePrice'], kde = True, hist=True, fit = norm)
plt.title('SalePrice distribution vs Normal Distribution', fontsize = 13)
plt.xlabel("House's sale Price in $", fontsize = 12)
plt.legend(['actual price dist','Normal dist ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],loc='best')
plt.show()

In [ ]:
(mu, sigma) = norm.fit(np.log(train_data['SalePrice']))
plt.figure(figsize = (12,6))
sns.distplot(np.log(train_data['SalePrice']), kde = True, hist=True, fit = norm)
plt.title('Log of SalePrice distribution vs Normal Distribution', fontsize = 13)
plt.xlabel("House's sale Price in log $", fontsize = 12)
plt.legend(['Log price dist','Normal dist ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],loc='best')
plt.show()

**Recommendation: Use Log-Transform on SalesPrice for Linear Models since it increases the accuracy of the normality assumption.**


## Consolidated Recommendations

- Recommendation: Considering the extreme changes by neighborhood, we recommend using means and modes at the neighborhood level when considering imputing missing values.  **done**  
- Recommendation: Drop or change id==1877 value for YearRemodAdd to 2002 or add a flat indictor variable for Remodeled **done**  
- Recommendation: Either drop id==2593 entry, change to YearBuilt to match the house, change to 2007, or replace with neighborhood mean.  **unneeded, its test data**  
- Recommendation: Drop 2819 **unneeded, its test data**  
    - There is one entry where the total areas of each component sum greater than the total lot - more square footage of buildings(plus) than the lot.
- Id 2039 LotFrontage and LotArea seem complex and likely is the result of an error. Recommendation: Remove this entry as an outlier.  **unneeded, its test data**  
- Recommendation: Use Log-Transform on SalesPrice for Linear Models since it increases the accuracy of the normality assumption.  **done**  



## Final Recommendations  

**Id** - Drop this feature because it does not add any value to the model.  
**GarageYrBlt** - Drop feature value typo 2207 outside of possible range (ie. >2010).  

---  

Rename these features to allow method calls such as "data.FirstFlrSF" and  
avoid issues with "data.1stFlrSF" not working.  
**1stFlrSF** - rename to FirstFlrSF  
**2ndFlrSF** - rename to SecondFlrSF   
**3SsnPorch** - rename to ThreeSsnPorch  

---  

These features had values that did not match the data descriptions, but do  
not have any affect on the current model. These could be changed to stay  
consistent if deemed necessary and to avoid errors on future test data.  
**MSZoning** - typo  (ie. description = 'C', actual = 'C (all)')  
**Neighborhood** - typo  (ie. description = 'Names', actual = 'NAmes')    
**BldgType** - typo  (ie. description = '2FmCon', 'TwnhsI', 'Duplx'  
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$
actual = '2fmCon', 'Twnhs',  'Duplex')   
**Exterior2nd** - typo  (ie. description = 'CemntBd', 'WdShing', 'BrkComm  
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ actual = 'CmentBd', 'Wd Shng', 'Brk Cmn')  

---

The following features were missing values from the data description  
because they did not appear in the training or testing data, but could be  
added to avoid erros on future test data.  
**MSZoning** - missing 'RP', 'C', 'A', 'I'  
**Utilities** - missing 'ELO', 'NoSewr'  
**Condition2** - missing 'RRNe'  
**OverallCond** - missing '10'  
**Exterior1st** - missing 'Other', 'PreCast'  
**Exterior2nd** - missing 'PreCast'  
**MasVnrType** - missing 'CBlock'  
**ExterQual** - missing 'Po'  


# Cleaning

## Drop or Replace Values

In [ ]:
# Recommendation: Replace id==1877 value for YearRemodAdd to 2002
data.loc[(data.Id == 1877),'YearRemodAdd'] = 2002

# Recommendation: Either drop id==2593 entry, change to YearBuilt to match the house, change to 2007, or replace with neighborhood mean.
data.loc[(data.Id == 2593),'GarageYrBlt'] = np.nan

In [ ]:
# drop extreme outliers in training data
data = data[~data.Id.isin([1298, 523, 1100, 533])]

## Drop Features

The Id column is artificially correlated and will skew results.

In [ ]:
# drop ID 
data.drop(['Id'],axis=1,inplace=True)

## Rename Columns

In [ ]:
data.rename(columns = {'1stFlrSF':'FirstFlrSF', '2ndFlrSF':'SecondFlrSF', '3SsnPorch': 'ThreeSsnPorch' }, inplace = True) 

## Replace NaNs

In [ ]:
# flag for creating means and modes by neighborhood rather than the entire dataset
by_neighborhood = True

# replace NaN
features_mean = ['LotFrontage', 'GrLivArea'] 

features_mode = ['MSZoning', 'Utilities', 'Exterior1st','Exterior2nd','MasVnrArea', 'BsmtFinSF1',
                 'BsmtFullBath','BsmtHalfBath','KitchenQual','SaleType','Electrical']

# replace nans with 'None'
features_none = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
                 'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

# replace nans with 0.0
features_zero = ['BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea']

# replace 'GarageYrBlt' with 'YearBuilt'
data['GarageYrBlt'].fillna(data['YearBuilt'], inplace=True)

features_spec = ['Functional']

for vector in features_mean:

  if by_neighborhood:
    # good code for neighborhood value-grabbing based on mean
    replace_value = data.groupby('Neighborhood')[vector].transform('mean')
    data[vector].fillna(replace_value, inplace=True)
  
  else:
    # code for data-wide mean grabbing as opposed to neighborhood specific
    mean = data[vector].astype("float").mean(axis=0)
    data[vector].replace(np.nan, mean, inplace = True)

for vector in features_mode:
  if by_neighborhood:
    # good code for neighborhood value-grabbing based on mode
    data[vector] = data.groupby('Neighborhood')[vector].transform(lambda x: x.fillna(x.mode().iloc[0]))
  
  else:
    # code for data-wide mean grabbing as opposed to neighborhood specific
    mode = data[vector].value_counts().idxmax()
    data[vector].replace(np.nan, mode, inplace = True)

for vector in features_none:
  data[vector].replace(np.nan, 'None', inplace = True)

for vector in features_zero:
  data[vector].replace(np.nan, 0.0, inplace = True)

for vector in features_spec:
  data[vector].replace(np.nan, 'Typ', inplace = True)



# Analysis

##Data Info
Below is a summary of the dataset's columns, non-null count, and dtype.

In [ ]:
data.info()

## Heatmap
The heatmap confirms that all Nans have been replaced.  The missing values of the right of the heatmap are the sales prices from the test data.

In [ ]:
plt.figure(figsize=(24,8))
sns.heatmap(data.isnull(),yticklabels=False,cbar=False)
plt.show()

## Correlation Matrix
The correlation matrix shows that ten features have a 0.50 or higher correlation with SalePrice.  These variables listed in order from highest correlation to lowest are: OverallQual, GrLivArea, GarageCars, GarageArea, TotalBsmtSF, FirstFlrSF, TotRmsAbvGrd, YearBuilt, and YearRemodAdd.  The overall material and finish of the house is by far the highest correlated feature with sales price at 0.79.

Features garage car capacity and garage square feet have the highest correlation at 0.89, which is not surprising.  Other notable features that are highly correlated are total rooms above ground and above grade living area square feet, year home built and year garage built, and first-floor square feet and basement square feet.  

Unexpectedly, the overall condition of the house does not have a high correlation with the sales price.  However, after further data exploration, it is determined that the ratings for overall condition are concentrated around five.  From the graph below, it is seen that overall quality has a more normal distribution than overall condition.  Therefore, it is understood why the home's overall condition has a low correlation with the sales price. 

In [ ]:
# Plot set up for correlation matrix
rc = {'font.size': 10, 'axes.labelsize': 16, 'legend.fontsize': 16, 'xtick.labelsize': 16, 'ytick.labelsize':16, }
plt.rcParams.update(rc)

In [ ]:
# Correlation Matrix
f, ax = plt.subplots(figsize=(28, 28))
mat = data.corr('pearson')
mask = np.triu(np.ones_like(mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
corr = sns.heatmap(mat, mask=mask, cmap=cmap, vmax=1, center=0, annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}).set_title('Correlation Matrix', fontsize=24)

plt.show()

### OverallQual and OverallCond Distributions

In [ ]:
# Plot set up
fig, axes = plt.subplots(1,2, figsize=(20,8))
style.use('ggplot')

# OverallQual Distirbution
overallqual_count = data['OverallQual'].value_counts() 
sns.barplot(overallqual_count.index, overallqual_count.values, ax=axes[0])
axes[0].set_title('Overall Quality Distribution')
axes[0].set_ylabel('Counts')
axes[0].set_xlabel('Rating')

# OverallCond Distribution
overallcond_count = data['OverallCond'].value_counts() 
sns.barplot(overallcond_count.index, overallcond_count.values, ax=axes[1])
axes[1].set_title('Overall Condition Distribution')
axes[1].set_ylabel('Counts')
axes[1].set_xlabel('Rating')

plt.show()

## Exploring OverallQual and GrLivArea

In [ ]:
# OverallQuall - SalePrice - GrLivArea
fig, axes = plt.subplots(1,3, figsize=(28,8))

fig.suptitle("Sales Price, Overall Quality, Above Grade Living Area")
sns.set_style("whitegrid")

# Box plot of OverallQual, SalePrice, and GrLivArea
sns.boxplot(data=train_data,x='OverallQual',y='SalePrice', hue=pd.cut(train_data["GrLivArea"], 4), ax=axes[0])
axes[0].set_title("Boxplot")

# Strip plot of OverallQual, SalePrice, and GrLivArea
sns.stripplot(data=train_data,x='OverallQual',y='SalePrice', hue=pd.cut(train_data["GrLivArea"], 4), ax=axes[1])
axes[1].set_title("Strip Plot")

# Violin plot of OverallQual, SalePrice, and GrLivArea
sns.violinplot(data=train_data,x='OverallQual',y='SalePrice', hue=pd.cut(train_data["GrLivArea"], 4), ax=axes[2])
axes[2].set_title("Violin Plot")


plt.show()

In [ ]:
# Reset rcParams to default 
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
# CONFIRM ALL NA VALUES ARE GONE (SALE PRICE SHOULD BE == 1459)
# data.isnull().sum(0).sort_values(ascending=False)

## Column Data Types

In [ ]:
  num_colums = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 
  'GarageCars', 'GarageArea', 'SalePrice', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'FirstFlrSF', 'SecondFlrSF', 'LowQualFinSF', 'GrLivArea',
  'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
  'ThreeSsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

  cat_columns=['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold', 'YrSold', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 
              'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
              'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
              'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 
              'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

  for col in cat_columns:
      data[col] = data[col].astype('object')

In [ ]:
# Turn all variables (columns) into dummy variables
def onehot_encoding(df, categorical_columns):

  valid_cat_columns = []
  # create new df
  new_df = df

  # remove columns that are not in cat_columns
  for col in categorical_columns:
    if col in df.columns: 
      valid_cat_columns.append(col)
  
  for i, col in enumerate(valid_cat_columns):

      # create dummies from column
      temp_df = pd.get_dummies(data[col], prefix=col, prefix_sep='_')
      
      # drop column from df
      df.drop([col],axis=1,inplace=True)

      # for first column, create new DF
      if i==0:
          new_df = temp_df.copy()

      # for subsequent columns, concat DFs
      else:
        new_df = pd.concat([new_df,temp_df],axis=1)

  # concat DF (with original columns not provided in columns list) with the dummy variables    
  new_df=pd.concat([df,new_df],axis=1)
      
  return new_df

# Split data (train, dev, test)

Use this line to get fresh data for each model, split into train, dev, test:


```
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, False)
```

Otherwise, just use

```
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, True)
```
to get train, test data to create a submission file.


In [ ]:
def get_data(data_frame, submission_flag=submission):
  all_data_frame = data_frame.copy()
  
  # Set the randomizer seed so results are the same each time.
  np.random.seed(0)

  # onehot_encode dataframe based off previously-created cat_columns 
  # this ensures that if a dataframe with features removed was passed into this function
  # only those columns still in the dataframe are encoded
  all_data_frame = onehot_encoding(all_data_frame, cat_columns)

  # Seperate data
  train_data = all_data_frame.loc[(all_data_frame['SalePrice'].notnull())]
  test_data = all_data_frame.loc[(all_data_frame['SalePrice'].isna())]

  # remove SalePrice from Train and dev data
  train_vectors = train_data.drop('SalePrice', axis = 1)

  # create label vector for train and dev data
  train_SalePrice = train_data['SalePrice']

  # remove SalePrice from Test data
  test_vectors = test_data.drop('SalePrice', axis = 1)

  # if submission_file is ON, create  ONLY train and test sets
  if submission_flag:
    return train_vectors, train_SalePrice, test_vectors

  # if submission_file is OFF, create train, dev, test sets
  else:
    # Create filter to split train and dev data (80% train, 20% dev)
    msk = np.random.rand(len(train_data)) < 0.8

    # remove SalePrice from Train and dev data
    train_vectors = train_data[msk].drop('SalePrice', axis = 1)
    dev_vectors = train_data[~msk].drop('SalePrice', axis = 1)

    # create label vector for train and dev data
    train_SalePrice = train_data[msk]['SalePrice']
    dev_SalePrice = train_data[~msk]['SalePrice']

    # create label vector for train and dev data
    train_SalePrice = train_data[msk]['SalePrice']

  return train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors

# Models

## Linear Model:

### Linear Model: All features

In [ ]:
submission = True

In [ ]:
if submission:
  train_vectors, train_SalePrice, test_vectors = get_data(data, True)

  reg = LinearRegression()
  reg.fit(train_vectors, np.log(train_SalePrice))
  y_pred = reg.predict(test_vectors)
  submission_file(np.exp(y_pred), 'linear_submission_log.csv')

train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, False)
# Linear Regression, all
reg = LinearRegression()
reg.fit(train_vectors, np.log(train_SalePrice))

y_pred = reg.predict(dev_vectors)

print(f" Score {reg.score(dev_vectors, np.log(dev_SalePrice))}")
print(f" MSE {sklearn.metrics.mean_squared_error(np.exp(y_pred), dev_SalePrice)}")
print(f" RMSE {np.sqrt(sklearn.metrics.mean_squared_error(np.exp(y_pred), dev_SalePrice))}")

#### Linear Model with all Features
Rank: 5979    Score:  15623.551

In [ ]:
# Plot set up
plt.style.use('ggplot')
plt.figure(figsize=(10,8))

# Set x and y variable
x, y = y_pred, np.log(dev_SalePrice)

# Scatter plot - fitted vs. actuals
plt.scatter(x, y, alpha=.7,
            color='b') #alpha helps to show overlapping data

# Line of best fit
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b)

# Plot set up
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.suptitle("Linear Regression Model", fontsize=16)
plt.title('All Features')
plt.xlim(10.5,15), plt.ylim(10.5,15)
plt.text(12.48,14.25,'Slope={}'.format(round(m,2)), fontsize=12)
plt.show()

#### Outlier Discussion
Several apparent outliers were apparent in the first run.  With this information, we elected to drop IDs [1298, 523, 1100, 533] from the training data.

Lets attempt to handle with some python libraries

[Cited Work](https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/)



In [ ]:
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, False)

In [ ]:
mae = 1

# summarize the shape of the updated training dataset
print('Original Shapes: ', train_vectors.shape, train_SalePrice.shape)

while mae > .097:
  # Show Mean Abs Error for original model
  mae = sklearn.metrics.mean_absolute_error(np.log(dev_SalePrice), y_pred)
  
  X_train, y_train = train_vectors, train_SalePrice
  X_test, y_test = dev_vectors, dev_SalePrice

  # identify outliers in the training dataset
  iso = IsolationForest(contamination=.09)
  yhat = iso.fit_predict(X_train)
  mask = yhat != -1

  X_train, y_train = X_train[mask], y_train[mask]

  # fit the model
  model = LinearRegression()
  model.fit(X_train, np.log(y_train))

  # evaluate the model
  yhat = model.predict(X_test)

  # evaluate predictions
  mae = sklearn.metrics.mean_absolute_error(np.log(y_test), yhat)

  print('Processed MAE: %.3f' % mae)

# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)

# evaluate the model
print(f" Score {model.score(X_test, np.log(y_test))}")
print(f" MSE {sklearn.metrics.mean_squared_error(yhat, np.log(dev_SalePrice))}")
print(f" RMSE {np.sqrt(sklearn.metrics.mean_squared_error(np.exp(yhat), dev_SalePrice))}")

if submission:

  # evaluate the model
  y_pred = model.predict(test_vectors)
  submission_file(np.exp(y_pred), 'isolation_forest_log.csv')

#### Linear Model with all Features and outliers removed
Rank: 2002    Score:  14223.31 ***Most optimal model***

In [ ]:
# Plot set up
plt.style.use('ggplot')
plt.figure(figsize=(10,8))

# Set x and y variable
x, y = yhat, np.log(y_test)

# Scatter plot - fitted vs. actuals
plt.scatter(x, y, alpha=.7,
            color='b') #alpha helps to show overlapping data

# Line of best fit
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b)

# Plot set up
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.suptitle("Linear Regression Model", fontsize=16)
plt.title('All Features')
plt.xlim(10.5,15.5), plt.ylim(10.5,15.5)
plt.text(12.48,14.25,'Slope={}'.format(round(m,2)), fontsize=12)
plt.show()

### Linear Model: Selective Features

In [ ]:
# Correlation Matrix
corr = data.corr()
highest_corr_features = corr.index[abs(corr["SalePrice"])>0.5] 
plt.figure(figsize=(10,10))
g = sns.heatmap(data[highest_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
df = data[highest_corr_features]
df["OverallQual"] = data["OverallQual"]

train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(df, False)

# Linear Regression, all
reg = LinearRegression()
reg.fit(train_vectors, np.log(train_SalePrice))

y_pred = reg.predict(dev_vectors)
print(reg.score(dev_vectors, np.log(dev_SalePrice)))
print(sklearn.metrics.mean_squared_error(y_pred, np.log(dev_SalePrice)))
print(np.sqrt(sklearn.metrics.mean_squared_error(y_pred, np.log(dev_SalePrice))))

X_train, y_train, X_test, y_test = train_vectors, train_SalePrice, dev_vectors, dev_SalePrice

# Show Mean Abs Error for original model
mae = sklearn.metrics.mean_absolute_error(np.log(y_test), y_pred)
print('Original MAE: %.3f' % mae)

# summarize the shape of the updated training dataset
print('Original Shapes: ', train_vectors.shape, train_SalePrice.shape)

# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)
mask = yhat != -1

X_train, y_train = X_train[mask], y_train[mask]

# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)

# fit the model
model = LinearRegression()
model.fit(X_train, np.log(y_train))

# evaluate the model
yhat = model.predict(X_test)

# evaluate predictions
mae = sklearn.metrics.mean_absolute_error(np.log(y_test), yhat)
print('MAE: %.3f' % mae)

# evaluate the model
yhat = model.predict(test_vectors)
submission_file(np.exp(yhat), 'linear_regression_selective.csv')

In [ ]:
np.exp(.14)

In [ ]:
# Plot set up
plt.style.use('ggplot')
plt.figure(figsize=(10,8))

# Scatter plot - fitted vs. actual
plt.scatter(y_pred, np.log(dev_SalePrice), alpha=.7,
            color='b') #alpha helps to show overlapping data

# Line of best fit
x, y = y_pred, np.log(dev_SalePrice).to_numpy()
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b)

# Plot set up
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.suptitle("Linear Regression Model", fontsize=16)
plt.title('Selective Features')
plt.xlim(10.5,15), plt.ylim(10.5,15)
plt.text(12.48,14.25,'Slope={}'.format(round(m,2)), fontsize=12)
plt.show()

In [ ]:
sns.pairplot(data=data,
                  y_vars=['SalePrice'],
                  x_vars=df.drop(['SalePrice'],axis=1,inplace=False).columns)

new_df = df

new_df.TotalBsmtSF = np.log(new_df.TotalBsmtSF)

sns.pairplot(data=data,
                  y_vars=['SalePrice'],
                  x_vars=df.drop(['SalePrice'],axis=1,inplace=False).columns)
plt.show()

## Lasso:


In [55]:
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, False)

# generate a range of alpha
alpha=np.arange(0.01, 1, 0.0005)
score_best = 0
i_best = 0
# initiate a lasso model using different alpha
for i in alpha:
  las = Lasso(alpha=i)
  las_fit = las.fit(train_vectors, np.log(train_SalePrice))
  score = las.score(dev_vectors, np.log(dev_SalePrice))
  # pick up the best score and its coresponding alpha
  if score > score_best:
    score_best = score
    alpha_best = i
# print out the best alpha and its score
# print(alpha_best,score_best)
print('score:',score_best)

# predict on dev data
las_best = Lasso(alpha=alpha_best)
las_best_fit = las_best.fit(train_vectors, np.log(train_SalePrice))
las_pred = las_best.predict(dev_vectors)
# print out the MSE and RMSE
print('MSE:',sklearn.metrics.mean_squared_error(las_pred, np.log(dev_SalePrice)))
print('RMSE:',np.sqrt(sklearn.metrics.mean_squared_error(las_pred, np.log(dev_SalePrice))))

if submission:

  train_vectors, train_SalePrice, test_vectors = get_data(data, True)

  las_best_fit = las_best.fit(train_vectors, np.log(train_SalePrice))
  y_pred = las_best.predict(test_vectors)
  submission_file(np.exp(y_pred), 'lasso_log.csv')

score: 0.8602606719733411
MSE: 0.021325342283049654
RMSE: 0.1460319906152404


In [56]:
# print out the number of features selected by the model
selected_feat = train_vectors.columns[(SelectFromModel(las_best,prefit=True).get_support())]
print('total features: {}'.format((train_vectors.shape[1])))
print('selected features: {}'.format(len(selected_feat)))

total features: 349
selected features: 26


## Enet:


In [57]:
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, False)

# generate a range of alpha
alpha=np.arange(0.01, 1, 0.0005)
score_best = 0
i_best = 0
# initiate a ENet model using different alpha
for i in alpha:
  enet = ElasticNet(alpha=i)
  enet_fit = enet.fit(train_vectors, np.log(train_SalePrice))
  score = enet.score(dev_vectors, np.log(dev_SalePrice))
  # pick up the best score and its coresponding alpha
  if score > score_best:
    score_best = score
    alpha_best = i
# print out the best alpha and its score
# print(alpha_best,score_best)
print('score:',score_best)

# predict on dev data
enet_best = ElasticNet(alpha=alpha_best)
enet_best_fit = enet_best.fit(train_vectors, np.log(train_SalePrice))
enet_pred = enet_best.predict(dev_vectors)
# print out the MSE and RMSE
print('MSE:',sklearn.metrics.mean_squared_error(enet_pred, np.log(dev_SalePrice)))
print('RMSE:',np.sqrt(sklearn.metrics.mean_squared_error(enet_pred, np.log(dev_SalePrice))))

if submission:

  train_vectors, train_SalePrice, test_vectors = get_data(data, True)

  net_best_fit = enet_best.fit(train_vectors, np.log(train_SalePrice))
  y_pred = enet_best.predict(test_vectors)
  submission_file(np.exp(y_pred), 'enet_log.csv')

score: 0.8873165583633741
MSE: 0.017196397009112994
RMSE: 0.13113503349262925


## XGBoost:

In [ ]:
regressor=xgboost.XGBRegressor()

booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

## Hyper Parameter Optimization
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

# Set up the random search with 4-fold cross validation
random_cv = GridSearchCV(estimator=regressor,
            param_grid=hyperparameter_grid,
            cv=5, 
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True
            )

if submission:
  # comment out when not fitting model ***TAKES A WHILE***
  random_cv.fit(train_vectors, train_SalePrice)
  filename = 'xgboost_model.pkl'
  pickle.dump(regressor, open(filename, 'wb'))



Fitting 5 folds for each of 3200 candidates, totalling 16000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   23.5s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 11.2min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 26.9min


In [ ]:
sns.distplot(data['FirstFlrSF'])

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

data['FirstFlrSF'] = boxcox1p(data['FirstFlrSF'], boxcox_normmax(data['FirstFlrSF'] + 1))

In [ ]:
sns.distplot(data['FirstFlrSF'])

# Fitting model (Advanced approach)

In [ ]:
train_vectors, dev_vectors, train_SalePrice, dev_SalePrice, test_vectors = get_data(data, True)


In [ ]:
def overfit_reducer(df):
    """
    This function takes in a dataframe and returns a list of features that are overfitted.
    """
    overfit = []
    for i in df.columns:
        counts = df[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df) * 100 > 99.94:
            overfit.append(i)
    overfit = list(overfit)
    return overfit


overfitted_features = overfit_reducer(data)

X = X.drop(overfitted_features, axis=1)
X_sub = X_sub.drop(overfitted_features, axis=1)

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)



In [ ]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [ ]:
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, 
                                              alphas=alphas2, 
                                              random_state=42, 
                                              cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


In [ ]:
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                             

In [ ]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )


In [ ]:
xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)


In [ ]:
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)


In [ ]:
# score = cv_rmse(stack_gen)
# print("Stack: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(ridge)
print("Ridge: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("LASSO: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

# score = cv_rmse(gbr)
# print("gbr: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )


# Conclusion

Considerable EDA was required in this project, with a fair amount of feature engineering as well.  Although there is certainly room to address distribution and skewness of additional features, we are quite satisfied with our linear model regression techniques.  Although we only ranked ~ 2000, because of the way Kaggle scores only half the test set prior to the close of the competition, we feel confident a good portion of the higher-ranked models will suffer from overfitting.

Overall, we included six models in our project：LinearRegression, LR with outlier removed, LR with features with correlation above 0.5, Lasso and Enet, as well as XGBoost. 

Additional model comparison with pros and cons can be found in the associated slide deck.
